# 사람의 몸짓을 읽어보자
크게 2차원에서 (x,y) 좌표를 찾는 기술 / 3차원에서 (x,y,z) 좌표를 찾는 기술로 나뉨<br>
하지만 2차원 이미지에서 3차원으로 복원하는 것은 굉장히 어려운 작업


---

## 개념 1. 영상 좌표계(Coordinate System)
> [영상 Geometry #1 좌표계](https://darkpgmr.tistory.com/77)


영상 geometry에서는 크게 4가지 좌표계가 존재함


![4 coordinate system](coordinate_system.png)
* 2차원 좌표계
 * 정규 좌표계 (Normalized Image Coordinate System)
 * 픽셀 좌표계 (Pixel Image Coordinate System)
* 3차원 좌표계
 * 카메라 좌표계 (Camera Coordinate System)
 * 월드 좌표계 (World Coordinate System)<br><br>
 
 
 ### 3차원 좌표계: (1) 월드 좌표계
문제에 대해 사람이 임의로 잡아서 사용하는 좌표계.<br>
예를 들어 자신의 방 한쪽 모서리를 원점으로 잡고 각 벽면을 X,Y,Z축으로 삼을 수 있음.<br>
> $ P=(X,Y,Z)$ 로 표기<br><br>


 ### 3차원 좌표계: (2) 카메라 좌표계
카메라를 기준으로 한 좌표계.<br>
카메라의 초점(렌즈의 중심)을 원점, 카메라의 정면 광학축 방향을 Z축, 카메라 아래쪽 방향을 Y축, 오른쪽 방향을 X축으로 삼음
![camera coordinate system](camera_coordinate_system.png)
> $P_c=(X_c,Y_c,Z_c)$ 로 표기<br><br>


 ### 2차원 좌표계: (1) 픽셀 좌표계(영상 좌표계)
 왼쪽 상단 끝(left-top)을 (0,0)으로 해서 x,y 좌표를 결정. 이 평면을 이미지 평면(image plane)이라고 부름.<br>
 단위는 픽셀(pixel).<br>
 3차원의 P(X,Y,Z)가 사진을 촬영하면서 $P_{img}$로 투영되는 것으로, 이 작업을 복원하려면 부가적인 정보 없이는 불가능함.
 > $P_{img}=(x,y)$
 
 
 ### 2차원 좌표계: (2) 정규 좌표계
 카메라의 내부 패러미터(intrinsic parameter)의 영향을 제거한 이미지 좌표계. 내부 패러미터란 3차원 영상이 2차원으로 투영될 때 계산하는 패러미터를 의미.<br>
 > [카메라 내부 패러미터](https://darkpgmr.tistory.com/32)
 
 
 좌표계 단위를 없앤**(정규화된)** 좌표계로, 카메라 초점과의 거리가 1인 가상의 이미지 평면을 정의함.<br>
 * 정규 좌표계에서 원점은 image plane 평면의 중점임(동시에 Z축과의 교점). 따라서 픽셀 좌표계와 중점이 다름.
 * 픽셀 이미지에서 '카메라 내부 패러미터를 이용해 역으로 추산하여 정규 이미지로 복원'하는 과정을 진행하면서 3차원 이미지로 복원하는 것.<br><br>
 
 
 ### == 정규 좌표계에서 픽셀 좌표계로 옮기는 과정의 예시 ==
> * $ p'=(u,v)$ : 정규 좌표계의 원점 (픽셀 좌표계의 x,y축과 구분하기 위해서 u,v라는 기호로 표기함)<br>
> 이를 픽셀 좌표계 $P_{img}$로 변환하려면 카메라 내부 패러미터를 이용하면 됨.
> * $ p_{img} = Kp' $<br> 이를 식으로 표현하면 <br>
> $\begin{bmatrix} x \\ y \\ 1\end{bmatrix} $ = $\begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix} $ $\begin{bmatrix} u \\ v \\ 1\end{bmatrix} $<br>
($f_x, f_y$는 초점 거리, $c_x, c_y$는 주점(광학축과 영상평면이 만나는 픽셀 좌표))



---

## 개념 2. homogeneous coordinate(projective coordinate)
간단히 말하면 __(x, y)를 (x, y, 1)__로 표현하는 것.<br>
더 일반화해서 말하면 __임의의 0이 아닌 상수 w에 대해 (x, y)를 (wx, wy, w)로 표현__하는 것.<br>
즉 __homogeneous 좌표계__에서 __스케일(scale)은 무시__되며 __(x, y)에 대한 homogeneous 좌표 표현은 무한히 많이 존재__<br><br>

* 마찬가지로, 3차원의 경우에는 (X, Y, Z)를 (X, Y, Z, 1) 나 (wX, wY, wZ, w)로 표현.
* 역으로 homogeneous 좌표에서 원래의 좌표를 구하려면 끝 자리가 1이 되도록 scale을 바꾼 후 1을 떼내면 됨.<br>
예를 들어 (x, y, α)라는 좌표가 있다면, (x/α, y/α, 1)로 scaling한 뒤 1을 떼면 실제 2차원 좌표는 (x/α, y/α).<br>
* (u, v) 방향으로 무한대의 점은 homogeneous 좌표로 (u, v, 0)으로 표현함.<br>

homogeneous 좌표 (x, y, z)에서 (x/z, y/z)를 구하는 것은 projection, 반대로, (u, v)를 homogeneous 좌표 (wu, wv, w)로 표현하는 것은 inverse projection 과정<br>


---

## 개념 3. Epipolar Geometry
2-view 관점에서의 기하를 다루는 것.<br>
동일한 사물 또는 대상에 관한 정보를 **서로 다른 두 지점**에서 획득했을 때, 영상 A와 영상 B의 매칭 사이의 기하학적 관계를 다루는 것.<br><br>


### == Epipolar Geometry 예시 ==
![epipolar geometry](epipolar.png)


위 그림에서 3차원 공간상의 한 점 P가 영상 A에서는 p에 투영되고, 영상 B에서는 p'에 투영됐다고 가정하면
* 두 카메라 원점을 잇는 선과 이미지 평면이 만나는 점 $e, e'$가 **epipole**.
* 투영점과 epipole을 잇는 직선 $l, l'$을 **epiline**(또는 epipole line)이라 부름.<br><br>


#### *그렇다면. 두 카메라 위치 사이의 기하학적 관계 [R|t]를 알고 있고, 영상 A의 영상 좌표 p를 알고 있다.<br>         이 때 영상 B의 영상 좌표 p'는 무엇일까?


참고로 P까지의 거리(depth) 정보를 모른다면 영상좌표 p에서 투영되기 전의 3차원 좌표 P를 복원할 수는 없음.<br>
따라서 점 p' 역시 알 수 없음.<br>
하지만 점 P는 A 카메라 원점에서 p를 잇는 직선(ray) 상에 존재하기 때문에, 이 직선을 영상 B에 투영시키면 그 **투영한 직선 어딘가에 p'가 있음**은 알 수 있음.<br>
이 투영된 직선이 바로 epiline l'<br><br>


#### 즉 이 문제에서 알 수 있는 것은, 영상좌표 p'은 하나로 답을 낼 수 없지만 p'이 지나는 직선인 epiline l'은 유일한 하나로 결정할 수 있다는 점이다.<br>


* 참고로 다른 영상으로 대응되는 epiline을 계산해주는 변환행렬이 바로 Fundamental Matrix, Essential Matrix임.
> [Fundamental Matrix, Essential Matrix 설명](https://darkpgmr.tistory.com/83?category=460965)


---

## 개념 4. 포즈를 찾는 접근법
![top-down, bottom-up](pose_detection.png)


#### Top-down
* 모든 사람의 정확한 keypoint 를 찾기 위해 object detection 을 사용.
* crop 한 이미지 내에서 keypoint 를 찾아냄.
* detector가 선행되어야 하고 모든 사람마다 알고리즘을 적용해야 하기 때문에 사람이 많이 등장할 때는 느림.<br><br>


#### Bottom-up
* detector가 없이 keypoint 를 먼저 검출.
* 예를 들어 손목에 해당하는 모든 점들을 검출.
* 이후 한 사람에 해당하는 keypoint를 clustering.
* detector가 없기 때문에 다수의 사람이 영상에 등장하더라도 속도 저하가 크지 않지만, keypoint 검출 범위가 넓어 top-down보다 성능이 떨어짐.<br><br>


#### 난점
기본적으로 face landmark 검출과 비슷하지만, 손이 얼굴을 가리는 행위, 모든 keypoint 가 영상에 담기지 않는 등 invisible , occlusions, clothing, lighting change와 같이 고려해야 할 요소가 많음.<br>
이러한 부분은 "keypoint 가 존재할 확률" 을 학습하면서 성능이 향상됨.

---
## 개념 5. 다양한 알고리즘
* Convolutional Pose Machines
![CPM](CPM.png)


Stage 1 은 image feature 를 계산하는 역할을 하고 stage 2는 keypoint 를 예측.<br>
g1과 g2 모두 heatmap 을 출력하게 만들어서 재사용이 가능한 부분은 weight sharing을 함.<br><br>


* Stacked Hourglass Network
![Stacked Hourglass Network](SHN.png)


feature map 크기가 작아졌다 커지는 구조여서 hourglass 라고 표현<br>
Conv layer 와 pooling 으로 이미지(또는 feature) 를 incoding 하고, upsampling layer 를 통해 feature map 의 크기를 키우는 방향으로 decoding.<br><br>


* SimpleBaseline
![SimpleBaseline](SB.png)


간단한 encoder-decoder 구조 모델을 만들어서 구성.


---

## 1. 실습: SimpleBaseline 모델

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

Metal device set to: Apple M2


2022-09-23 17:13:07.584730: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-23 17:13:07.584955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### deconv 모듈 만들기.

In [2]:
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

모듈에서 중복을 제거

In [3]:
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [4]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

#### 각각의 요소를 합쳐 모델을 완성

In [5]:
inputs = keras.Input(shape=(256, 192, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 192, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 sequential (Sequential)     (None, 64, 48, 256)       10489600  
                                                                 
 conv2d (Conv2D)             (None, 64, 48, 17)        4369      
                                                                 
Total params: 34,081,681
Trainable params: 34,027,025
Non-trainable params: 54,656
_________________________________________________________________


#### 가상의 이미지를 넣어서 출력이 되는지 확인

In [6]:
np_input = np.zeros((1,256,192,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=tf.float32)
print('input shape')
print (tf_input.shape)
print('\n')

tf_output = model(tf_input)
print('output shape')
print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

input shape
(1, 256, 192, 3)


output shape
(1, 64, 48, 17)
tf.Tensor(
[[[-7.03176856e-03  2.24632933e-03  3.09681427e-03 -1.13343121e-03
   -2.55749817e-03 -3.46572464e-03  5.63131471e-04 -1.34674669e-03
    1.53942744e-03 -1.94370211e-03]
  [ 9.76333115e-03 -3.29062645e-03 -1.08789485e-02  2.48795468e-03
   -3.95793235e-03  1.69575414e-05  1.80640176e-03 -1.02248024e-02
    4.92259115e-03  9.44276154e-03]
  [ 3.31660477e-03  6.07968029e-03 -9.27487054e-06 -1.00478726e-02
   -1.52959237e-02  2.09875638e-03  5.79849235e-04 -1.04613611e-02
    5.51425572e-03  8.32019188e-03]
  [-4.85235779e-03  4.15649312e-03  8.85251351e-03 -7.55378231e-03
    3.55566922e-03  8.89973063e-03  8.69012997e-03  4.98279463e-03
   -8.39812867e-03  1.32432636e-02]
  [-1.19966932e-03  5.70248207e-03  6.00640522e-03 -4.69959865e-04
   -1.08566657e-02 -1.79107171e-02  3.35779879e-03  7.39747379e-03
   -1.07576279e-02  1.05755217e-02]
  [-6.43373583e-04 -2.35338067e-03 -9.14032664e-03 -1.12941172e-02
   -1.119532